# 杭州萧山区项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-07-02*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [3]:
chem = xr.open_dataset('F:/Data/Project_Xiaoshan/202209/COMBINE_ACONC_v54_ISAM_intel_CN3XS_135X138_2022_chem.nc')
grid = xr.open_dataset('F:/Data/Project_Xiaoshan/GRIDCRO2D_2022234.nc')

In [4]:
# 将时间点转换为实际时间
times=pd.date_range(timestart,timeend,freq='h')
times

DatetimeIndex(['2022-09-01 00:00:00', '2022-09-01 01:00:00',
               '2022-09-01 02:00:00', '2022-09-01 03:00:00',
               '2022-09-01 04:00:00', '2022-09-01 05:00:00',
               '2022-09-01 06:00:00', '2022-09-01 07:00:00',
               '2022-09-01 08:00:00', '2022-09-01 09:00:00',
               ...
               '2022-09-30 14:00:00', '2022-09-30 15:00:00',
               '2022-09-30 16:00:00', '2022-09-30 17:00:00',
               '2022-09-30 18:00:00', '2022-09-30 19:00:00',
               '2022-09-30 20:00:00', '2022-09-30 21:00:00',
               '2022-09-30 22:00:00', '2022-09-30 23:00:00'],
              dtype='datetime64[ns]', length=720, freq='H')

In [5]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [6]:
# 结合地形高度HT与各气压层离地高度ZH获得各层实际海拔高度
ht=np.squeeze(grid.HT)
height=np.zeros([np.size(chem.ZH,0),
                 np.size(chem.ZH,1),
                 np.size(chem.ZH,2),
                 np.size(chem.ZH,3)])
for t in range(np.size(chem.ZH,0)):
    for l in range(np.size(chem.ZH,1)):
        height[t,l,:,:]=chem.ZH[t,l,:,:]+ht

In [7]:
days=2 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from CMAQ
        CO=(['time','level','y','x'],chem.CO[days*24-8:-8,:,:,:].data*30/22.4,{'long name':'Carbon Monoxide','units':'ug m-3'}),
        O3=(['time','level','y','x'],chem.O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Ozone','units':'ug m-3'}),
        NO=(['time','level','y','x'],chem.NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Nitric Oxide','units':'ug m-3'}),
        NO2=(['time','level','y','x'],chem.NO2[days*24-8:-8,:,:,:].data*37/22.4,{'long name':'Nitrogen Dioxide','units':'ug m-3'}),
        SO2=(['time','level','y','x'],chem.SO2[days*24-8:-8,:,:,:].data*64/22.4,{'long name':'Sulfur Dioxide','units':'ug m-3'}),
        VOC=(['time','level','y','x'],chem.VOC[days*24-8:-8,:,:,:].data,{'long name':'Volitile Organic Compounds','units':'ppbC'}),
        PM25=(['time','level','y','x'],chem.PM25_TOT[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        PM10=(['time','level','y','x'],chem.PM10[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        # ! wwind
        wwind=(['time','level','y','x'],chem.WWind[days*24-8:-8,:,:,:].data,{'long name':'Vertical Wind Speed','units':'m s-1'}),
        # ! altitude
        HT=(['time','level','y','x'],height[days*24-8:-8,:,:,:].data,{'long name':'Altitudes','units':'m'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Xiaoshan_202209',
        grid='CN3XS_135X138',
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 720, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2022-09-01 ... 2022-09-30T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 28.29 28.29 28.29 28.29 ... 31.89 31.89 31.89
    longitude  (y, x) float32 117.9 117.9 117.9 118.0 ... 122.3 122.3 122.4
Dimensions without coordinates: y, x
Data variables:
    CO         (time, level, y, x) float32 334.1 327.5 325.9 ... 47.37 47.34
    O3         (time, level, y, x) float32 97.6 96.9 ... 1.022e+03 1.026e+03
    NO         (time, level, y, x) float32 0.04603 0.01541 ... 3.891e-09
    NO2        (time, level, y, x) float32 14.2 13.2 12.87 ... 0.1209 0.121
    SO2        (time, level, y, x) float32 6.652 6.392 ... 0.006858 0.006853
    VOC        (time, level, y, x) float32 51.58 51.8 52.74 ... 0.02283 0.02279
    PM25       (time, level, y, x) float32 19.68 19.33 19.22 ... 0.0 0.0 0.0
    PM10       (time, level, y, x) float32 21.87 21.46 21.28 ... 0.0 0.0 0.0
    wwind      (time, level, y, x) float32 0.02919 0.04026 ... -0.001054
    HT         (time, level, y, x) float64 115.6 203.4 ... 1.963e+04 1.963e+04
Attributes:
    case:     Xiaoshan_202209
    grid:     CN3XS_135X138

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/XS_2209_chem.nc',encoding=encoding)